# Import PycroMagnetics

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import sys
import os

path = os.getcwd()
path = os.path.dirname(path)
path = os.path.dirname(path)
sys.path.append(path)

import PycroMagnetics.pymag as ppm
import PycroMagnetics.mag_visualize as pmv
from PycroMagnetics import magmatlib as mml

# Define Physical Space

In [63]:
# Define space
m1 = ppm.Ferromagnet(Lx=50*10**-7, 
                     Ly=50*10**-7, 
                     Lz=1*10**-7, 
                     nx=50, 
                     ny=50, 
                     nz=1,
                     PBCx=False,
                     PBCy=False,
                     PBCz=False)

# Set geometry
m1.setCylindricalMask(center_x=25*10**-7, 
                      center_y=25*10**-7, 
                      center_z=0.5*10**-7, 
                      radius_x=25*10**-7, 
                      radius_y=25*10**-7, 
                      height=1*10**-7,
                      angle=0)

# Get material
mater1 = mml.Ni_PolyCrystal()
mater1.Ms = 1080

# Asign material to the object
m1.asign_material(mater1)

# Initialize magnetization state
m1.setUniformMagnetization(thetaM0=90, phiM0=0)

# Print asigned property
# print('mx'); pmv.show_MagProperties(m1.mx, vmin=-1, vmax=1, figsize=(5,5), fontsize=15)
# print('my'); pmv.show_MagProperties(m1.my, vmin=-1, vmax=1, figsize=(5,5), fontsize=15)
# print('mz'); pmv.show_MagProperties(m1.mz, vmin=-1, vmax=1, figsize=(5,5), fontsize=15)

# Warm up evolution

In [64]:
evol= ppm.Evolver(magnet=m1,
                  tstep=1e-13,
                  warm_up=True,
                  uniaxial_anisotropy_field=False,
                  exchang_field=True,
                  demag_field=True,
                  external_field=True,
                  DMI_field=False,
                  FLT_field=False,
                  DLT_field=False,
                  thermal_field=True,
                  random_state=None)

Start calculating demag factors
End calculating demag factors


In [65]:
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [66]:
evol.tstep=5e-13

In [ ]:
# Set total calculation step number
total = 1000000

# Set verbosity
verbose = 1

# Set Logger
mlogger = ppm.MagnetLogger(log_name='new')

# Set uniform external magnetic field
evol.setUniformExternalMagneticField(Hextx=0, Hexty=0, Hextz=0)

# Set start time
start = time.perf_counter()

# Start calculation loop
for i in range(total):

    # Calculate one step of evolution
    evol.evolve()
    
    # Set optimized time step
#     evol.cal_tstep(ideal_dm=0.0001)
    
    # Get end time
    end = time.perf_counter()
    
    # Get log
    if i%1000 == 0:
        total_mx = evol.magnet.mx.sum()
        total_my = evol.magnet.my.sum()
        total_mz = evol.magnet.mz.sum()
        mask_area = evol.magnet.mask.sum()
        mlogger(time=evol.time, dm=evol.dm, 
                mean_mx=total_mx/mask_area, 
                mean_my=total_my/mask_area, 
                mean_mz=total_mz/mask_area)
    
    # Save evolver
    if i%5000 == 0:
        ppm.save_Evolver(evol, 'test1')
        
    if i%1000 == 0:   
        if verbose > 0:
            print('{}/{} time:{}'.format(i,total, end - start))
            print('current dm & tstep : {:e} & {:e}'.format(evol.dm, evol.tstep))
            print('  mean (mx, my, mz) : ({:e}, {:e}, {:e})'.format(total_mx/mask_area, 
                                                                    total_my/mask_area,
                                                                    total_mz/mask_area))

            
    # Set early stop
    if i==0:
        endure = 0
    if evol.dm<1.0*10**-15:
        endure += 1
    if evol.dm>=1.0*10**-15:
        endure = 0
    if endure >= 10:
        break

0/1000000 time:0.0092847999994774
current dm & tstep : 3.511908e-03 & 5.000000e-13
  mean (mx, my, mz) : (9.999993e-01, -7.665360e-06, 2.191348e-06)
1000/1000000 time:8.279010899999776
current dm & tstep : 4.318044e-03 & 5.000000e-13
  mean (mx, my, mz) : (9.992801e-01, 5.459698e-04, 7.375111e-06)
2000/1000000 time:16.537655299999642
current dm & tstep : 4.189744e-03 & 5.000000e-13
  mean (mx, my, mz) : (9.991767e-01, 9.773436e-04, -7.416417e-05)
3000/1000000 time:24.833839500000067
current dm & tstep : 4.437878e-03 & 5.000000e-13
  mean (mx, my, mz) : (9.991837e-01, 1.110482e-03, 1.970190e-04)
4000/1000000 time:32.91130239999984
current dm & tstep : 4.683988e-03 & 5.000000e-13
  mean (mx, my, mz) : (9.991839e-01, 6.559513e-04, 1.187011e-04)
5000/1000000 time:41.116954399999486
current dm & tstep : 4.647238e-03 & 5.000000e-13
  mean (mx, my, mz) : (9.991681e-01, 1.678938e-04, -2.561469e-04)
6000/1000000 time:49.395759
current dm & tstep : 3.418726e-03 & 5.000000e-13
  mean (mx, my, mz)